In [ ]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM, Merge
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

In [ ]:
#parameters
maxlen = 30
labels = 2

In [ ]:
input = pd.read_csv("/Users/apple1/Desktop/project542/all/train.csv")
input.columns = ['first', 'last','b_or_n']

# remove encode 
input['first'] = input['first'].str[2:-1]
input['last'] = input['last'].str[2:-1]

input['firstlen']= [len(str(i)) for i in input['first']]
input['lastlen'] = [len(str(i)) for i in input['last']]
input1 = input[(input['firstlen'] >= 2) & (input['lastlen'] >= 2)]

In [ ]:
firsts = input['first']
lasts = input['last']
labels = input['b_or_n']

vocab = set(' '.join([str(i) for i in firsts]))
vocab = set(' '.join([str(i) for i in lasts]))

vocab.add('END')
len_vocab = len(vocab)

In [ ]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [ ]:
#train test split 
msk = np.random.rand(len(input1)) < 0.8

train = input1[msk]
test = input1[~msk]

In [ ]:
def set_flag(i):
    tmp = np.zeros(39);
    tmp[i] = 1
    return(tmp)

In [ ]:
# Truncating and padding training data
train_X = []
train_Y = []
train_Z = []

trunc_train_first = [str(i)[0:maxlen] for i in train['first']]
trunc_train_last = [str(i)[0:maxlen] for i in train['last']]

for i in trunc_train_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)

for i in trunc_train_last:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_Y.append(tmp)
    
for i in train['b_or_n']:
    if i == 1:
        train_Z.append([1,0])
    else:
        train_Z.append([0,1])


In [ ]:
print(train_X[1])

In [ ]:
np.asarray(train_X).shape

In [ ]:
np.asarray(train_Y).shape

In [ ]:
np.asarray(train_Z).shape

In [ ]:
#build the model: 2 stacked LSTM
print('Building LSTM model')

left_branch = Sequential()
left_branch.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
right_branch = Sequential()
right_branch.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))

model = Sequential()
model.add(Merge([left_branch, right_branch], mode='concat'))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))

# Softmax activation function
model.add(Activation('softmax'))

# Cross-entropy loss, metric is accuracy
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
# Truncating and padding test data

test_X = []
test_Y = []
test_Z = []

trunc_test_first = [str(i)[0:maxlen] for i in test['first']]
trunc_test_last = [str(i)[0:maxlen] for i in test['last']]

for i in trunc_test_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)

for i in trunc_test_last:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_Y.append(tmp)
    
for i in test['b_or_n']:
    if i == 1:
        test_Z.append([1,0])
    else:
        test_Z.append([0,1])

In [ ]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)
print(np.asarray(test_Z).shape)

In [ ]:
batch_size = 1000
model.fit([np.asarray(train_X), np.asarray(train_Y)], train_Z, batch_size=batch_size, nb_epoch=50, validation_data=([np.asarray(test_X), np.asarray(test_Y)], test_Z))

In [ ]:
score, acc = model.evaluate([np.asarray(test_X), np.asarray(test_Y)], test_Z)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
#save our model and data
model.save_weights('model', overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [ ]:
evals = model.predict([np.asarray(test_X), np.asarray(test_Y)])
prob_m = [i[0] for i in evals]

In [ ]:
out = pd.DataFrame(prob_m)
out['first'] = test['first'].reset_index()['first']
out['last'] = test['last'].reset_index()['last']
out['b_or_n'] = test['b_or_n'].reset_index()['b_or_n']

In [ ]:
out.head(10)
out.columns = ['prob_b','first', 'last', 'actual']
out.head(10)
out.to_csv("pred_out.csv")

In [ ]:
# small test. first 3 names are spanish and last 3 names are Brazilian 
first = ["manuel","luis","ezequil","michel","luiz","alvaro"]
last = ["garcia","sanchez","gomez","caetano","guilherme","lime"]
X = []
Y = []
trunc_first = [i[0:maxlen] for i in first]
trunc_last = [i[0:maxlen] for i in last]
for i in trunc_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
for i in trunc_last:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    Y.append(tmp)
    
pred = model.predict([np.asarray(X), np.asarray(Y)])
pred